In [2]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import os
import pandas as pd
import datetime

In [3]:
df_arg = pd.read_excel("20220427_農產品_分析.xlsx",sheet_name=0)
print(df_arg)

       作物       日期     價格
0      大豆  2019/05  67.00
1      大豆  2019/06  61.20
2      大豆  2019/07  52.77
3      大豆  2019/11  62.00
4      大豆  2019/12  70.67
...    ..      ...    ...
12143  蘿蔔  2021/07   5.10
12144  蘿蔔  2021/08   6.20
12145  蘿蔔  2021/10   5.46
12146  蘿蔔  2021/11   8.09
12147  蘿蔔  2021/12   7.54

[12148 rows x 3 columns]


In [4]:
df_weather = pd.read_excel("20220427_農產品_分析.xlsx",sheet_name=1)
print(df_weather)


           日期   測站  平均溫度  最高溫度  最高溫度日期  最低溫度  最低溫度日期  雨量(毫米)  最大十分鐘風速(m/s)  \
0     2009/12  阿里山   6.4  16.7      12  -0.6      20    73.7           4.2   
1     2009/12   鞍部  10.5  20.9       9   3.1      20   210.5          11.8   
2     2009/12   板橋  16.8  27.8       9  10.1      19    80.4           6.6   
3     2009/12   成功  19.6  27.1      11  14.1      21    30.8           9.1   
4     2009/12   嘉義  17.5  30.2      15   7.7      22    18.2           7.4   
...       ...  ...   ...   ...     ...   ...     ...     ...           ...   
4484  2021/01   梧棲  14.8  21.2      16   7.3      13     1.5          17.6   
4485  2021/01   新屋  14.6  26.2      20   6.3       8    42.0          14.5   
4486  2021/01   宜蘭  15.3  25.2      21   7.0       8    46.0           8.7   
4487  2021/01   玉山  -0.6  17.9      21 -14.4       9    18.5          14.8   
4488  2021/01  竹子湖  11.0  24.8      21   1.6       8   347.0           8.0   

     最大十分鐘風向(360°)  ...  最大瞬間風日期  平均相對溼度  最小相對溼度  最小相對溼度日期  氣壓(

In [68]:
list_str = ["測站","測站地區"]
list_float = []
for i in df_weather.columns:
    if "日期" not in i and "測站" not in i and "地區" not in i:
        list_float.append(i)
        
for j in list_float:
    check_list = df_weather[[j]].values
    for i in check_list.tolist():

        if not isinstance(i[0], float):
            try:
                float(i[0])
            except:
                print("在%s表格中" % (j))
                print(i[0])
                print(type(i[0]))
#2013年6月台北最大十分鐘風向(360°)表格中為Ｖ，2020/8 彭佳嶼在最小相對溼度表格中為X，2019/8基隆在氣壓(百帕)表格中為Y

在最大十分鐘風向(360°)表格中
V
<class 'str'>
在最小相對溼度表格中
X
<class 'str'>
在氣壓(百帕)表格中
V
<class 'str'>


In [52]:
from scipy import stats
import time

file_name = "20220427_農產品_分析.xlsx"

df_arg = pd.read_excel(file_name,0)
df_weather = pd.read_excel(file_name,1)

In [51]:
#平均溫度
#計算進行了幾筆資料
n = 1
#x應為天氣，而y是價格，如果顯著就可以說，可以透過天氣來預測價格
for agriculture in np.unique(df_arg[["作物"]].values):
    #找出該作物的所有日期
    month = df_arg[(df_arg.作物 == agriculture)][["日期"]].values
    for location in np.unique(df_weather[["測站"]].values):
        x = pd.DataFrame()
        #篩選出吻合作物日期的天氣
        for m in month:
            locat = df_weather[(df_weather.日期 == m[0]) & (df_weather.測站 == location)][["平均溫度"]]
            x = x.append(locat)
        y = df_arg[(df_arg.作物 == agriculture)][["價格"]]
        #將x,y的資料變成1d array
        x = list(np.array(x).flatten())
        y = list(np.array(y).flatten())
        if len(x) != len(y):
            print("%s的資料不足，請刪除" % location)
        #線性回歸的資料
        slope, intercept, r, p, std_err = stats.linregress(x, y)
        if p < 0.05 and r**2 > 0.9:
            print("%s 的平均溫度可預測 %s的價格" % (location, agriculture))
            print("P value: %f" % (p))
            print("R: %f" % (r*r))
            print('intercept:',intercept)
            print('slope:',slope)
            print("-----------------------------------")
    #檢驗資料進度
    print("第%d筆資料已完成 - %s ,共%d筆資料"% (n, agriculture, len(np.unique(df_arg[["作物"]].values))))
    print("-----------------------------------")
    n += 1


第1筆資料已完成 - 冬瓜 ,共145筆資料
-----------------------------------
第2筆資料已完成 - 加工麻竹筍 ,共145筆資料
-----------------------------------
第3筆資料已完成 - 咖啡(櫻桃豆) ,共145筆資料
-----------------------------------
第4筆資料已完成 - 咖啡(水洗生豆) ,共145筆資料
-----------------------------------
第5筆資料已完成 - 咖啡(生豆) ,共145筆資料
-----------------------------------
第6筆資料已完成 - 大芥菜(鮮食) ,共145筆資料
-----------------------------------
玉山 的平均溫度可預測 大豆的價格
P value: 0.000071
R: 0.908265
intercept: 90.92709562934513
slope: -4.277543876707667
-----------------------------------
第7筆資料已完成 - 大豆 ,共145筆資料
-----------------------------------
第8筆資料已完成 - 大豆(黑豆) ,共145筆資料
-----------------------------------
第9筆資料已完成 - 小番茄(玉女) ,共145筆資料
-----------------------------------
第10筆資料已完成 - 小白菜 ,共145筆資料
-----------------------------------
第11筆資料已完成 - 小蕃茄(聖女) ,共145筆資料
-----------------------------------
第12筆資料已完成 - 山藥 ,共145筆資料
-----------------------------------
第13筆資料已完成 - 巨峰葡萄 ,共145筆資料
-----------------------------------
第14筆資料已完成 - 幸水梨 ,共145筆資料
-------------------------

第130筆資料已完成 - 青香蕉(內銷) ,共145筆資料
-----------------------------------
第131筆資料已完成 - 韭菜 ,共145筆資料
-----------------------------------
第132筆資料已完成 - 韭黃 ,共145筆資料
-----------------------------------
第133筆資料已完成 - 香瓜(美濃瓜) ,共145筆資料
-----------------------------------
第134筆資料已完成 - 香菇(太空包)乾 ,共145筆資料
-----------------------------------
第135筆資料已完成 - 香菇(太空包)鮮 ,共145筆資料
-----------------------------------
第136筆資料已完成 - 香蕉(內銷) ,共145筆資料
-----------------------------------
第137筆資料已完成 - 馬鈴薯(食用) ,共145筆資料
-----------------------------------
第138筆資料已完成 - 鮑魚菇 ,共145筆資料
-----------------------------------
第139筆資料已完成 - 鳳梨(17號) ,共145筆資料
-----------------------------------
第140筆資料已完成 - 鳳梨(開英) ,共145筆資料
-----------------------------------
第141筆資料已完成 - 鳳梨釋迦 ,共145筆資料
-----------------------------------
第142筆資料已完成 - 鶯歌桃 ,共145筆資料
-----------------------------------
第143筆資料已完成 - 麻竹筍 ,共145筆資料
-----------------------------------
第144筆資料已完成 - 龍眼(大粒) ,共145筆資料
-----------------------------------
第145筆資料已完成 - 龍眼(粉殼) ,共145筆資料
-------

In [56]:
#最大十分鐘風速(m/s)
#計算進行了幾筆資料
n = 1
#x應為天氣，而y是價格，如果顯著就可以說，可以透過天氣來預測價格
for agriculture in np.unique(df_arg[["作物"]].values):
    #找出該作物的所有日期
    month = df_arg[(df_arg.作物 == agriculture)][["日期"]].values
    for location in np.unique(df_weather[["測站"]].values):
        x = pd.DataFrame()
        #篩選出吻合作物日期的天氣
        for m in month:
            locat = df_weather[(df_weather.日期 == m[0]) & (df_weather.測站 == location)][["最大十分鐘風速(m/s)"]]
            x = x.append(locat)
        y = df_arg[(df_arg.作物 == agriculture)][["價格"]]
        #將x,y的資料變成1d array
        x = list(np.array(x).flatten())
        y = list(np.array(y).flatten())
        if len(x) != len(y):
            print("%s的資料不足，請刪除" % location)
        #線性回歸的資料
        slope, intercept, r, p, std_err = stats.linregress(x, y)
        if p < 0.05 and r**2 > 0.9:
            print("%s 的平均溫度可預測 %s的價格" % (location, agriculture))
            print("P value: %f" % (p))
            print("R: %f" % (r*r))
            print('intercept:',intercept)
            print('slope:',slope)
            print("-----------------------------------")
    #檢驗資料進度
    print("第%d筆資料已完成 - %s ,共%d筆資料"% (n, agriculture, len(np.unique(df_arg[["作物"]].values))))
    print("-----------------------------------")
    n += 1
    time.sleep(10)



第1筆資料已完成 - 冬瓜 ,共145筆資料
-----------------------------------
第2筆資料已完成 - 加工麻竹筍 ,共145筆資料
-----------------------------------
第3筆資料已完成 - 咖啡(櫻桃豆) ,共145筆資料
-----------------------------------
第4筆資料已完成 - 咖啡(水洗生豆) ,共145筆資料
-----------------------------------
第5筆資料已完成 - 咖啡(生豆) ,共145筆資料
-----------------------------------
第6筆資料已完成 - 大芥菜(鮮食) ,共145筆資料
-----------------------------------
第7筆資料已完成 - 大豆 ,共145筆資料
-----------------------------------
第8筆資料已完成 - 大豆(黑豆) ,共145筆資料
-----------------------------------
第9筆資料已完成 - 小番茄(玉女) ,共145筆資料
-----------------------------------
第10筆資料已完成 - 小白菜 ,共145筆資料
-----------------------------------
第11筆資料已完成 - 小蕃茄(聖女) ,共145筆資料
-----------------------------------
第12筆資料已完成 - 山藥 ,共145筆資料
-----------------------------------
第13筆資料已完成 - 巨峰葡萄 ,共145筆資料
-----------------------------------
第14筆資料已完成 - 幸水梨 ,共145筆資料
-----------------------------------
第15筆資料已完成 - 扁蒲 ,共145筆資料
-----------------------------------
第16筆資料已完成 - 敏豆(四季豆) ,共145筆資料
-----------------------------------
第17

第133筆資料已完成 - 香瓜(美濃瓜) ,共145筆資料
-----------------------------------
第134筆資料已完成 - 香菇(太空包)乾 ,共145筆資料
-----------------------------------
第135筆資料已完成 - 香菇(太空包)鮮 ,共145筆資料
-----------------------------------
第136筆資料已完成 - 香蕉(內銷) ,共145筆資料
-----------------------------------
第137筆資料已完成 - 馬鈴薯(食用) ,共145筆資料
-----------------------------------
第138筆資料已完成 - 鮑魚菇 ,共145筆資料
-----------------------------------
第139筆資料已完成 - 鳳梨(17號) ,共145筆資料
-----------------------------------
第140筆資料已完成 - 鳳梨(開英) ,共145筆資料
-----------------------------------
第141筆資料已完成 - 鳳梨釋迦 ,共145筆資料
-----------------------------------
第142筆資料已完成 - 鶯歌桃 ,共145筆資料
-----------------------------------
第143筆資料已完成 - 麻竹筍 ,共145筆資料
-----------------------------------
第144筆資料已完成 - 龍眼(大粒) ,共145筆資料
-----------------------------------
第145筆資料已完成 - 龍眼(粉殼) ,共145筆資料
-----------------------------------
